In [74]:
import pandas as pd
import numpy as np
import os

DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), "data")

df = pd.read_excel(os.path.join(DATA_PATH, "Nordic_Textile_Anatomy_Database_DdS.xlsx"), sheet_name="RMM_DK")


In [75]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

INSIGNIFICANT_FIBER_THRESHOLD = 0.01 # Minimum percentage for fibers to be not disregarded

def cluster_composition_by_category(df, fiber_cols=3, min_clusters=2, max_clusters=4):
    """
    Clusters textiles by fiber composition within each Category.
    
    Parameters:
    - df: DataFrame with columns:
        'Category',
        'Fibre 1', 'Fibre 1 % Range', ..., up to 'Fibre {fiber_cols}', 'Fibre {fiber_cols} % Range'
    - fiber_cols: number of fiber columns to consider (default 3)
    - min_clusters, max_clusters: range of k to try for KMeans
    
    Returns:
    - dict mapping category to summary DataFrame with columns:
        'Cluster', 'Count', and one column per fiber name giving average percentage in that cluster.
    """
    # Helper to parse percentage range to midpoint
    def parse_pct(s):
        try:
            s = str(s).replace('–', '-').replace('%', '')
            low, high = s.split('-')
            return (float(low) + float(high)) / 2
        except:
            return np.nan

    # Collect all unique fiber names from columns Fibre 1 .. Fibre fiber_cols
    fiber_names = set()
    parsed_entries = []
    for idx, row in df.iterrows():
        comp = {}
        for i in range(1, fiber_cols + 1):
            name = row.get(f'Fibre {i}')
            pct_range = row.get(f'Fibre {i} % Range')
            if pd.notna(name) and pd.notna(pct_range):
                pct = parse_pct(pct_range)
                if pd.notna(pct):
                    comp[name] = pct
                    fiber_names.add(name)
        parsed_entries.append(comp)

    all_fibers = sorted(fiber_names)
    if not all_fibers:
        print("No fiber data found. Check column names and data.")
        return {}

    # Build feature matrix: each row is normalized composition vector over all_fibers
    feature_rows = []
    indices = []
    categories = []
    for (idx, row), comp in zip(df.iterrows(), parsed_entries):
        if comp:
            vec = [comp.get(f, 0.0) for f in all_fibers]
            total = sum(vec)
            if total > 0:
                vec = [v / total for v in vec]
                feature_rows.append(vec)
                indices.append(idx)
                cat = row['Category'].strip().lower()
                categories.append(cat)
    if not feature_rows:
        print("No valid composition entries to cluster.")
        return {}

    feat_df = pd.DataFrame(feature_rows, index=indices, columns=all_fibers)
    result = {}

    # Group by category
    cat_series = pd.Series(categories, index=indices, name='Category')
    for cat, group in cat_series.groupby(cat_series):
        idxs = group.index
        X = feat_df.loc[idxs]
        n_samples = len(X)
        if n_samples < 2:
            print(f"Category '{cat}' has fewer than 2 samples, skipping.")
            continue

        # Determine best k by silhouette
        best_k = None
        best_score = -1
        for k in range(min_clusters, min(max_clusters, n_samples - 1) + 1):
            km = KMeans(n_clusters=k, random_state=0)
            labels = km.fit_predict(X)
            # Silhouette requires at least 2 clusters and less than n_samples clusters
            score = silhouette_score(X, labels)
            if score > best_score:
                best_score = score
                best_k = k

        km = KMeans(n_clusters=best_k, random_state=0).fit(X)
        labels = km.labels_
        centroids = km.cluster_centers_

        # Build summary for this category
        summary = []
        for cluster_label in range(best_k):
            mask = labels == cluster_label
            count = int(mask.sum())
            centroid = centroids[cluster_label]
            # Convert centroid to shares
            pct = centroid / centroid.sum()
            comp_dict = {f: pct_val for f, pct_val in zip(all_fibers, pct)}
            row_summary = {'Cluster': cluster_label, 'Count': count}
            row_summary.update(comp_dict)
            summary.append(row_summary)

        summary_df = pd.DataFrame(summary).sort_values('Cluster').reset_index(drop=True)

        # drop insignificant fibers
        summary_df = summary_df.loc[:, summary_df.max() >= INSIGNIFICANT_FIBER_THRESHOLD]
        cols_to_clean = summary_df.select_dtypes(include='number').columns

        # Replace values < 0.01 with 0
        summary_df[cols_to_clean] = summary_df[cols_to_clean].where(summary_df[cols_to_clean] >= INSIGNIFICANT_FIBER_THRESHOLD, 0)

        # check to what percentage fibres add up
        meta_cols = ['Cluster', 'Count']
        fiber_cols = [col for col in summary_df.columns if col not in meta_cols]
        summary_df['Sum'] = summary_df[fiber_cols].sum(axis=1)

        result[cat] = summary_df

    return result

# Run clustering if df exists
if 'df' not in globals():
    print("Please ensure your DataFrame is named 'df' with columns 'Category', "
          "'Fibre 1'..'Fibre 3', 'Fibre 1 % Range'..'Fibre 3 % Range'.")
else:
    clusters = cluster_composition_by_category(df)
    for cat, summary_df in clusters.items():
        print(f"\nCategory: {cat}")
        display(summary_df)



Category: dresses and skirts


,Cluster,Count,Acetate,Acrylic,Cotton,Cupro,Flax/linen,Lyocell,Modal,Polyamide/nylon,Polyester,Silk,True Hemp,Viscose,Wool,Sum
0,0,101,0.000000,0.000000,0.011960,0.010881,0.015683,0.000000,0.000000,0.083617,0.034981,0.000000,0.000000,0.831314,0.000000,0.988436
1,1,144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.935601,0.000000,0.000000,0.036122,0.000000,0.971723
2,2,119,0.000000,0.000000,0.907988,0.000000,0.000000,0.000000,0.000000,0.016260,0.050928,0.000000,0.000000,0.000000,0.000000,0.975176
3,3,48,0.020833,0.025508,0.025743,0.000000,0.082156,0.178178,0.206947,0.190879,0.125604,0.083333,0.011448,0.000000,0.041667,0.992296



Category: handkerchiefs, ties, scarves, gloves and other


,Cluster,Count,Acrylic,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Cotton,Flax/linen,Other,Polyacrylate,Polyamide/nylon,Polyester,Polyurethane,Silk,Viscose,Wool,Sum
0,0,211,0.029313,0.000000,0.021832,0.000000,0.000000,0.000000,0.019834,0.898997,0.000000,0.000000,0.000000,0.000000,0.969976
1,1,115,0.000000,0.000000,0.963146,0.000000,0.000000,0.000000,0.000000,0.023660,0.000000,0.000000,0.000000,0.000000,0.986807
2,2,179,0.197992,0.049323,0.000000,0.013035,0.017409,0.010339,0.177141,0.046814,0.017997,0.102426,0.018295,0.316098,0.966870



Category: overcoats and anoraks


...

,Cluster,Count,Acrylic,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Cotton,Flax/linen,Lyocell,Modal,Polyamide/nylon,Polyester,Viscose,Wool,Sum
0,0,417,0.000000,0.000000,0.927468,0.00000,0.000000,0.012109,0.000000,0.052035,0.000000,0.000000,0.991612
1,1,92,0.013598,0.000000,0.164190,0.00000,0.000000,0.000000,0.013072,0.726647,0.072199,0.000000,0.989706
2,2,53,0.000000,0.000000,0.012250,0.03492,0.094340,0.018868,0.300721,0.035644,0.484352,0.000000,0.981094
3,3,18,0.022800,0.038956,0.016378,0.00000,0.085757,0.000000,0.028870,0.050750,0.000000,0.756489,1.000000



Category: trousers and shorts


,Cluster,Count,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Cotton,Elastane/Spandex,Elastane/spandex,Flax/linen,Lyocell,Modal,Other,Polyamide/nylon,Polyester,Polyurethane,Silk,Viscose,Wool,Sum
0,0,327,0.000000,0.904088,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065431,0.000000,0.000000,0.000000,0.000000,0.969520
1,1,45,0.014411,0.000000,0.016412,0.016067,0.014815,0.010642,0.000000,0.000000,0.896258,0.031395,0.000000,0.000000,0.000000,0.000000,1.000000
2,2,67,0.000000,0.026895,0.000000,0.000000,0.080455,0.055933,0.040185,0.014925,0.074299,0.045932,0.014925,0.014925,0.405990,0.217333,0.991798
3,3,155,0.000000,0.071444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.807205,0.000000,0.000000,0.076517,0.020081,0.975247



Category: underwear, socks, night clothes


,Cluster,Count,Acrylic,Cotton,Elastane/Spandex,Elastane/spandex,Flax/linen,Lyocell,Modal,Other,Polyamide/nylon,Polyester,Polypropylene,Viscose,Wool,Sum
0,0,130,0.000000,0.000000,0.031698,0.057349,0.000000,0.00000,0.000000,0.000000,0.880754,0.000000,0.000000,0.000000,0.00000,0.969800
1,1,214,0.000000,0.833637,0.000000,0.000000,0.000000,0.00000,0.000000,0.012632,0.078419,0.065211,0.000000,0.000000,0.00000,0.989898
2,2,84,0.000000,0.017317,0.000000,0.000000,0.011905,0.02381,0.035714,0.000000,0.059735,0.569411,0.000000,0.233678,0.00000,0.951570
3,3,51,0.094987,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.216753,0.000000,0.034445,0.000000,0.64505,0.991235


In [76]:
merged = []

for category, df_cat in clusters.items():
    df_cat = df_cat.copy()
    df_cat['Category'] = category  # Add category column
    merged.append(df_cat)

# Combine all into one DataFrame
merged_df = pd.concat(merged, ignore_index=True)

# Sort by Category (A-Z), then Count (descending)
merged_df = merged_df.sort_values(by=['Category', 'Count'], ascending=[True, False]).reset_index(drop=True)
merged_df = merged_df.fillna(0)
merged_df["Market share"] = merged_df["Count"] / merged_df["Count"].sum()
merged_df["Category share"] = merged_df["Count"] / merged_df.groupby("Category")["Count"].transform("sum")

cols = ['Category'] + [col for col in merged_df.columns if col != 'Category']

merged_df = merged_df[cols]
merged_df.head()


,Category,Cluster,Count,Acetate,Acrylic,Cotton,Cupro,Flax/linen,Lyocell,Modal,...,Other,Polyacrylate,Polyurethane,Polyimide,Elastane/Spandex,Elastane/spandex,Polyethylene,Polypropylene,Market share,Category share
0,dresses and skirts,1,144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032036,0.349515
1,dresses and skirts,2,119,0.000000,0.000000,0.907988,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026474,0.288835
2,dresses and skirts,0,101,0.000000,0.000000,0.011960,0.010881,0.015683,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022469,0.245146
3,dresses and skirts,3,48,0.020833,0.025508,0.025743,0.000000,0.082156,0.178178,0.206947,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010679,0.116505
4,"handkerchiefs, ties, scarves, gloves and other",0,211,0.000000,0.029313,0.021832,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046941,0.417822


In [77]:
merged_df.sort_values(by="Count", ascending=False,inplace=True)
merged_df.head()

,Category,Cluster,Count,Acetate,Acrylic,Cotton,Cupro,Flax/linen,Lyocell,Modal,...,Other,Polyacrylate,Polyurethane,Polyimide,Elastane/Spandex,Elastane/spandex,Polyethylene,Polypropylene,Market share,Category share
26,"t-shirts, singlets and vests, hoodies and crew...",0,417,0.0,0.000000,0.927468,0.0,0.0,0.0,0.012109,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.092770,0.718966
30,trousers and shorts,0,327,0.0,0.000000,0.904088,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.072747,0.550505
7,overcoats and anoraks,0,279,0.0,0.000000,0.028029,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.011143,0.0,0.0,0.0,0.0,0.0,0.062069,0.507273
34,"underwear, socks, night clothes",1,214,0.0,0.000000,0.833637,0.0,0.0,0.0,0.000000,...,0.012632,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.047608,0.446764
4,"handkerchiefs, ties, scarves, gloves and other",0,211,0.0,0.029313,0.021832,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.046941,0.417822


In [78]:
N_ROWS = 10 # Number of rows sorted by marketshare to export
FIBER_FILTER = 0.05 # Filter to keep only fibers where at least one clothing item has a fiber share above this value

# Select top 10 rows
top10 = merged_df.head(N_ROWS).copy(deep=True)
top10 = top10.fillna(0)

# Identify fibre columns (exclude meta columns)
meta_cols = ['Category', 'Sum', 'Cluster', 'Count', 'Market share', 'Category share']
fiber_cols = [col for col in top10.columns if col not in meta_cols]

# Filter out fibre columns where the max value is less than 5%
keep_fibers = [col for col in fiber_cols if top10[col].max(skipna=True) >= FIBER_FILTER]

# Keep only meta columns and filtered fibre columns
filtered_top10 = top10[meta_cols + keep_fibers]
#filtered_top10 = filtered_top10.drop(columns=['Cluster'])

# Add lifetime as another row based on WRAP 2022 data
# Rewrite lifetime_map as min/max tuple values
lifetime_map_minmax = {
    "dresses and skirts": (4.2, 4.9),
    "handkerchiefs, ties, scarves, gloves and other": (4.3, 4.3),
    "overcoats and anoraks": (5.4, 6.3),
    "shirts, blouses, tops": (4.1, 4.8),
    "sportswear and swimwear": (2.6, 4.4),
    "suits and blazers": (4.1, 6.1),
    "sweaters and cardigans": (4.0, 4.8),
    "t-shirts, singlets and vests, hoodies and crewnecks": (4.0, 4.0),
    "trousers and shorts": (3.8, 4.8),
    "underwear, socks, night clothes": (2.6, 4.4)
}

filtered_top10 = filtered_top10.rename(columns={'Sum': 'Fibre composition sum'})
# Map to new columns
filtered_top10['Lifetime Min'] = filtered_top10['Category'].map(lambda x: lifetime_map_minmax.get(x, (np.nan, np.nan))[0])
filtered_top10['Lifetime Max'] = filtered_top10['Category'].map(lambda x: lifetime_map_minmax.get(x, (np.nan, np.nan))[1])
filtered_top10 = filtered_top10[['Category'] + ['Category share'] + ['Market share'] + ['Lifetime Min'] + ['Lifetime Max']+ ['Count'] + ['Fibre composition sum'] + keep_fibers]
filtered_top10 


,Category,Category share,Market share,Lifetime Min,Lifetime Max,Count,Fibre composition sum,Acrylic,Cotton,Polyamide/nylon,Polyester,Silk,Viscose,Wool,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)"
26,"t-shirts, singlets and vests, hoodies and crew...",0.718966,0.092770,4.0,4.0,417,0.991612,0.000000,0.927468,0.000000,0.052035,0.000000,0.000000,0.000000,0.000000
30,trousers and shorts,0.550505,0.072747,3.8,4.8,327,0.969520,0.000000,0.904088,0.000000,0.065431,0.000000,0.000000,0.000000,0.000000
7,overcoats and anoraks,0.507273,0.062069,5.4,6.3,279,0.973859,0.000000,0.028029,0.000000,0.934687,0.000000,0.000000,0.000000,0.000000
34,"underwear, socks, night clothes",0.446764,0.047608,2.6,4.4,214,0.989898,0.000000,0.833637,0.078419,0.065211,0.000000,0.000000,0.000000,0.000000
4,"handkerchiefs, ties, scarves, gloves and other",0.417822,0.046941,4.3,4.3,211,0.969976,0.029313,0.021832,0.019834,0.898997,0.000000,0.000000,0.000000,0.000000
11,"shirts, blouses, tops",0.388119,0.043604,4.1,4.8,196,0.984087,0.000000,0.954704,0.013655,0.015728,0.000000,0.000000,0.000000,0.000000
15,sportswear and swimwear,0.594156,0.040712,2.6,4.4,183,0.992727,0.000000,0.000000,0.010920,0.914067,0.000000,0.000000,0.000000,0.000000
5,"handkerchiefs, ties, scarves, gloves and other",0.354455,0.039822,4.3,4.3,179,0.966870,0.197992,0.000000,0.177141,0.046814,0.102426,0.018295,0.316098,0.049323
12,"shirts, blouses, tops",0.348515,0.039155,4.1,4.8,176,0.971892,0.000000,0.026395,0.013810,0.931687,0.000000,0.000000,0.000000,0.000000
22,sweaters and cardigans,0.357602,0.037152,4.0,4.8,167,0.995527,0.312796,0.013653,0.209134,0.134409,0.000000,0.134517,0.064720,0.115999


In [79]:
filtered_top10['Market share'].sum()

0.5225806451612903

In [80]:
filtered_top10["Clothing type"] = [
    't-shirts', 
    'trousers',
    'overcoats',
    'underwear',
    'handkerchiefs_1',
    'shirts_1',
    'sportswear',
    'handkerchiefs_2',
    'shirts_2',
    'sweaters', 
]
# Move "clothing type" to the first column
cols = ['Clothing type'] + [col for col in filtered_top10.columns if col != 'Clothing type']
filtered_top10 = filtered_top10[cols]
filtered_top10

,Clothing type,Category,Category share,Market share,Lifetime Min,Lifetime Max,Count,Fibre composition sum,Acrylic,Cotton,Polyamide/nylon,Polyester,Silk,Viscose,Wool,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)"
26,t-shirts,"t-shirts, singlets and vests, hoodies and crew...",0.718966,0.092770,4.0,4.0,417,0.991612,0.000000,0.927468,0.000000,0.052035,0.000000,0.000000,0.000000,0.000000
30,trousers,trousers and shorts,0.550505,0.072747,3.8,4.8,327,0.969520,0.000000,0.904088,0.000000,0.065431,0.000000,0.000000,0.000000,0.000000
7,overcoats,overcoats and anoraks,0.507273,0.062069,5.4,6.3,279,0.973859,0.000000,0.028029,0.000000,0.934687,0.000000,0.000000,0.000000,0.000000
34,underwear,"underwear, socks, night clothes",0.446764,0.047608,2.6,4.4,214,0.989898,0.000000,0.833637,0.078419,0.065211,0.000000,0.000000,0.000000,0.000000
4,handkerchiefs_1,"handkerchiefs, ties, scarves, gloves and other",0.417822,0.046941,4.3,4.3,211,0.969976,0.029313,0.021832,0.019834,0.898997,0.000000,0.000000,0.000000,0.000000
11,shirts_1,"shirts, blouses, tops",0.388119,0.043604,4.1,4.8,196,0.984087,0.000000,0.954704,0.013655,0.015728,0.000000,0.000000,0.000000,0.000000
15,sportswear,sportswear and swimwear,0.594156,0.040712,2.6,4.4,183,0.992727,0.000000,0.000000,0.010920,0.914067,0.000000,0.000000,0.000000,0.000000
5,handkerchiefs_2,"handkerchiefs, ties, scarves, gloves and other",0.354455,0.039822,4.3,4.3,179,0.966870,0.197992,0.000000,0.177141,0.046814,0.102426,0.018295,0.316098,0.049323
12,shirts_2,"shirts, blouses, tops",0.348515,0.039155,4.1,4.8,176,0.971892,0.000000,0.026395,0.013810,0.931687,0.000000,0.000000,0.000000,0.000000
22,sweaters,sweaters and cardigans,0.357602,0.037152,4.0,4.8,167,0.995527,0.312796,0.013653,0.209134,0.134409,0.000000,0.134517,0.064720,0.115999


In [81]:
# Export merged_df as an Excel file
output_path = os.path.join(DATA_PATH, "clustered_fiber_composition.xlsx")
filtered_top10.to_excel(output_path, index=False)